# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Business Analytics</font>

# <font color='blue'>Supply Chain Analytics</font>
## <font color='blue'>Mini-Projeto 10</font>
### <font color='blue'>Sistema de Recomendação Para Rede de Varejo Usando Market Basket Analysis</font>

## Definição do Problema

Quer você faça compras com listas de compras meticulosamente
planejadas ou deixe que o capricho guie seus passos, nossos rituais únicos de
compra definem quem somos. Instacart, um aplicativo de pedido e entrega de
supermercado, tem como objetivo facilitar o preenchimento de sua geladeira e
despensa com seus itens pessoais favoritos e itens básicos quando você precisar
deles. Depois de selecionar produtos por meio do aplicativo Instacart, os
compradores revisam seus pedidos, fazem compras e a entrega é feita na loja
mais próxima a você.
A equipe de ciência de dados da Instacart desempenha um papel
importante no fornecimento dessa experiência de compra agradável. Atualmente,
eles usam dados transacionais para desenvolver modelos que preveem quais
produtos um usuário comprará novamente, quais tentará pela primeira vez ou
quais adicionará ao carrinho durante uma sessão. Recentemente, a Instacart
disponibilizou esses dados de forma aberta e o link para download você encontra
logo abaixo
Neste projeto de ciência de dados, você usará esses dados anônimos nos
pedidos dos clientes ao longo do tempo para prever quais produtos adquiridos
anteriormente estarão no próximo pedido de um usuário.

## Fonte de Dados

https://www.kaggle.com/c/instacart-market-basket-analysis/data

## Instalando e Carregando os Pacotes

In [1]:
!pip install fpgrowth_py

  Using cached fpgrowth_py-1.0.0-py3-none-any.whl (5.6 kB)


In [2]:
# Imports
from fpgrowth_py import fpgrowth
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Etapa 1 - Análise Exploratória

### Carregando os Dados

In [ ]:
# Carrega os dados
df_aisles = pd.read_csv("dados/aisles.csv", encoding = 'latin1')
df_departments = pd.read_csv("dados/departments.csv", encoding = 'latin1')
df_order_products__prior = pd.read_csv("dados/order_products__prior.csv", encoding = 'latin1')
df_order_products__train = pd.read_csv("dados/order_products__train.csv", encoding = 'latin1')
df_orders = pd.read_csv("dados/orders.csv", encoding = 'latin1')
df_products = pd.read_csv("dados/products.csv", encoding = 'latin1')
df_sample_submission = pd.read_csv("dados/sample_submission.csv", encoding = 'latin1')

In [ ]:
# Amostra de dados
df_aisles.head()

In [ ]:
df_aisles.info()

In [ ]:
# Amostra de dados
df_departments.head()

In [ ]:
df_departments.info()

In [ ]:
df_order_products__prior.info()

In [ ]:
df_order_products__train.info()

In [ ]:
#une os dois dataframes
df_order_products=df_order_products__prior.append(df_order_products__train)
len(df_order_products)

In [ ]:
df_order_products__prior.head()

In [ ]:
df_order_products__prior.info()

In [ ]:
df_order_products__train.head()

In [ ]:
df_order_products__train.info()

In [ ]:
df_orders.drop(['eval_set'],axis = 1, inplace=True)

In [ ]:
df_orders.head()

In [ ]:
df_orders.info()

In [ ]:
df_products.head()

In [ ]:
df_products.info()

In [ ]:
df_sample_submission.head()

In [ ]:
df_sample_submission.info()

In [ ]:
# Verificando valores ausentes
df_aisles.apply(lambda x: sum(x.isnull()))

In [ ]:
# Verificando valores ausentes
df_departments.apply(lambda x: sum(x.isnull()))

In [ ]:
# Verificando valores ausentes
df_order_products__prior.apply(lambda x: sum(x.isnull()))

In [ ]:
# Verificando valores ausentes
df_order_products__train.apply(lambda x: sum(x.isnull()))

In [ ]:
# Verificando valores ausentes
df_orders.apply(lambda x: sum(x.isnull()))

In [ ]:
# Verificando valores ausentes
df_products.apply(lambda x: sum(x.isnull()))

In [ ]:
# Verificando valores ausentes
df_sample_submission.apply(lambda x: sum(x.isnull()))

In [ ]:
df_departments.info()

In [ ]:
## Une todos os dataframes em um único

In [ ]:
df=df_aisles.merge(df_products,how='inner',on='aisle_id')
len(df)

In [ ]:
df=df.merge(df_departments,how='inner',on='department_id')
len(df)

In [ ]:
df=df.merge(df_order_products,how='inner',on='product_id')
len(df)

In [ ]:
df=df.merge(df_orders,how='inner',on='order_id')
len(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['days_since_prior_order']=df['days_since_prior_order'].fillna(0) #fill na values with 0

In [ ]:
df.isnull().sum()

In [ ]:
# Tipos de dados
df.info()

In [ ]:
df['days_since_prior_order']=df['days_since_prior_order'].astype(int)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# Matriz de correlação para as variáveis numéricas
fig, ax = plt.subplots(figsize = (24,12))         
sns.heatmap(df[['aisle', 'product_name', 'department', 'add_to_cart_order', 'reordered', 'order_number', 'order_dow', 'order_hour_of_day','days_since_prior_order']].corr(), annot = True, linewidths = .5, fmt = '.1g', cmap = 'Purples') 

> De acordo com a Matriz de Correlação, podemos ver que o existe uma corelação se o pedido foi reordenado e o número da ordem, embora não seja significativa.

### Quantidade de Produtos Vendidos por Departamento

In [ ]:
# Agrupa os dados por departamento
df_departament = df.groupby('department')

In [ ]:
# Plot
plt.figure(1)
df_departament['product_id'].sum().sort_values(ascending = False).plot.pie(figsize = (18,8), 
                                                                                  title = "Quantidade de Produtos Vendidos por Departamento")

Pelo gráfico acima podemos observar que o departamento produce é o que possui maior quantidade de unidades vendidas

### Quantidade de Produtos Vendidos por Hora do Dia

In [ ]:
# Agrupa por Hora do Dia
df_hour_of_day = df.groupby('order_hour_of_day')

In [ ]:
# Plot
plt.figure(1)
df_hour_of_day['product_id'].sum().sort_values(ascending = False).plot.barh(figsize = (18,8), 
                                                                                  color  = ['red'], 
                                                                                  title = "Quantidade de Produtos Vendidos por Hora do Dia")

No gráfico acima podemos ver que o periodo que apresenta maior quantidade de vendas são as 10h 

## Quantidade de Produtos Vendidos por Reordenamento

In [ ]:
# Agrupa por reordered
df_reordered = df.groupby('reordered')

In [ ]:
# Plot
plt.figure(1)
df_reordered['product_id'].sum().sort_values(ascending = False).plot.bar(figsize = (18,8), 
                                                                                  color  = ['yellow'], 
                                                                                  title = "Quantidade de Produtos Vendidos por Reordenamento")

Podemos notar que a maior parte dos produtos vendidos foram reordenados

### Média de Vendas Por Corredor

In [ ]:
# Agrupa os dados por corredor
df_corredor = df.groupby('aisle')

In [ ]:
# Plot
plt.figure(1)
df_corredor['product_id'].mean().sort_values(ascending = False).plot.barh(figsize = (20,20), 
                                                                                  color  = ['blue'], 
                                                                                  title = "Média de Vendas Por Corredor")

Pelo gráfico acima podemos verificar que o corredor que possui maior média de vendas é o de acessórios para bebês

## Total de Vendas Por Dias Desde o Último Pedido

In [ ]:
# Agrupa os dados por dias desde o ultimo pedido
df_days = df.groupby('days_since_prior_order')

In [ ]:
# Plot
plt.figure(1)
df_days['product_id'].sum().sort_values(ascending = False).plot.barh(figsize = (20,20), 
                                                                                  color  = ['green'], 
                                                                                  title = "Vendas Por Dias Desde o Último Pedido")

Pelo gráfico acima podemos verificar que a maioria das vendas foram realizadas 7 dias desde o último pedido

## Total de Vendas por Dia da Semana

In [ ]:
# Agrupa os dados por dias da semane
df_dow = df.groupby('order_dow')

In [ ]:
# Plot
plt.figure(1)
df_dow['product_id'].mean().sort_values(ascending = False).plot.pie(figsize = (20,20), title = "Total de Vendas por Dia da Semana")

Pelo gráfico acima podemos verificar que a maioria das vendas foram realizadas no domingo

In [ ]:
# Vamos salvar os dados processados até aqui
df.to_csv("dados/DataCoSupplyChainDatasetProcessados.csv")

## Etapa 2 - Regras de Associação

As Regras de Associação representam um dos conceitos mais importantes de aprendizado de máquina usado principalmente na análise de cesta de compras. Ou seja, precisamos de dados de vendas de produtos.

Em uma loja (ou portal de e-commerce), todos os vegetais são colocados no mesmo corredor, todos os laticínios são colocados juntos e os cosméticos formam outro conjunto desses grupos. 

Investir tempo e recursos em posicionamentos deliberados de produtos não apenas reduz o tempo de compra do cliente, mas também lembra o cliente de quais itens relevantes ele pode estar interessado em comprar, ajudando assim as lojas a fazerem vendas cruzadas no processo. Outra vantagem é na cadeia de suprimentos, pois os sistemas de entrega podem ser adaptados e personalizados ao padrão de compra dos clientes.

As Regras de Associação ajudam a descobrir todas essas relações entre itens de bancos de dados imensos. Uma coisa importante a se notar é que as Regras de Associação não extraem a preferência de um indivíduo, em vez disso, encontram relações entre um conjunto de elementos de cada transação distinta. Isso é o que os torna diferentes da filtragem colaborativa, por exemplo.

In [8]:
# Carregamos os dados processados e seguimos com o trabalho de análise
df = pd.read_csv("dados/DataCoSupplyChainDatasetProcessados.csv", encoding = 'latin1')

In [9]:
# Visualiza
df.tail()

,Unnamed: 0,aisle_id,aisle,product_id,product_name,department_id,department,order_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
33819101,33819101,100,missing,48811,General Tsao's Tofu,21,missing,2712617,1,0,175174,4,0,17,14.0
33819102,33819102,100,missing,48813,Lndbrg Wht Jasm Rice 8 Z,21,missing,1717868,1,0,7431,24,2,11,30.0
33819103,33819103,100,missing,48828,Popcorn Butter Flavor,21,missing,832621,1,0,75414,24,1,13,7.0
33819104,33819104,100,missing,49482,Black Tea With Vanilla Flavor In Pyramid Bags,21,missing,2868431,1,0,38108,39,3,9,4.0
33819105,33819105,100,missing,49553,Organic Plain Skyr,21,missing,1648925,1,1,9568,8,4,16,6.0


In [10]:
df.columns

Index(['Unnamed: 0', 'aisle_id', 'aisle', 'product_id', 'product_name',
       'department_id', 'department', 'order_id', 'add_to_cart_order',
       'reordered', 'user_id', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')

In [11]:
# quantidade de produtos únicos
len(df['product_id'].unique())

49685

In [12]:
# quantidade de corredores únicos
len(df['aisle'].unique())

134

In [13]:
# Vamos associar clientes e produtos
df_ = df.copy()
df_ = df_[['user_id', 'product_name']]
df_ = df_.drop_duplicates(['user_id', 'product_name'])
df_final= df_.groupby('user_id')['product_name'].apply(list).reset_index(name = "Products")

In [14]:
# Visualiza
df_final.head()

,user_id,Products
0,1,"[Organic String Cheese, Original Beef Jerky, P..."
1,2,"[Chicken Caesar Salad, Uncured Slow Cooked Ham..."
2,3,"[Original Rotisserie Chicken, Organic Shredded..."
3,4,"[Roasted Pine Nut Hummus, Real Guacamole, Yell..."
4,5,"[Uncured Genoa Salami, Semi-Soft Cheese, Ripen..."


In [15]:
#df_.to_csv("dados/df_final.csv")

In [16]:
#df_final=pd.read_csv("dados/df_final.csv")

In [17]:
# remove valores nulos
df_final['Products'].dropna(inplace=True)

In [18]:
df_final['Products'].isna().sum()

0

In [19]:
#df_final.drop(['Unnamed: 0'],axis=1,inplace=True)

In [20]:
itemSetList = []
for prod in df_final['Products']:
    itemSetList.append(prod)

In [21]:
len(itemSetList)

206209

### Aplicando o Algoritmo Frequent Pattern Mining

In [22]:
freqItemSet, rules = fpgrowth(itemSetList, minSupRatio=0.08, minConf=0.4)

In [23]:
print('Itens adquiquiridos frequentemente:')

Itens adquiquiridos frequentemente:


In [35]:
for item in freqItemSet:
    print(list(item))

['100% Whole Wheat Bread']
['Small Hass Avocado']
['Half & Half']
['Organic Half & Half']
['Boneless Skinless Chicken Breasts']
['Organic Granny Smith Apple']
['Sparkling Water Grapefruit']
['Grated Parmesan']
['Organic Garnet Sweet Potato (Yam)']
['Jalapeno Peppers']
['Organic Large Extra Fancy Fuji Apple']
['Raspberries']
['Red Onion']
['Organic Gala Apples']
['Original Hummus']
['Organic Red Bell Pepper']
['Red Vine Tomato']
['Bunched Cilantro']
['Organic Peeled Whole Baby Carrots']
['Michigan Organic Kale']
['Red Peppers']
['Organic Blackberries']
['Green Bell Pepper']
['Organic Tomato Cluster']
['Honeycrisp Apple']
['Organic Ginger Root']
['Apple Honeycrisp Organic']
['Organic Baby Arugula']
['Organic Whole Milk']
['Blueberries']
['Organic Italian Parsley Bunch']
['Organic Cucumber']
['Organic Small Bunch Celery']
['Fresh Cauliflower']
['Organic Fuji Apple']
['Carrots']
['Organic Red Onion']
['Asparagus']
['Extra Virgin Olive Oil']
['Organic Baby Carrots']
['Organic Cilantro']
['O

In [27]:
Antecedente=[]
Consequente=[]
Confiança=[]

for rule in rules:
    Antecedente.append(rule[0].pop())
    Consequente.append(rule[1].pop())
    Confiança.append(rule[2])
    
data={}
data['Antecedente']=Antecedente
data['Consequente']=Consequente
data['Confiança']=Confiança

data_final=pd.DataFrame(data)

### Ordenando por ordem decrescente em nível de confiança

In [29]:
data_final.sort_values(['Confiança'],ascending=False,inplace=True)
data_final.reset_index(inplace=True)
data_final.drop('index',axis=1,inplace=True)

### Regras de Associação

In [36]:
data_final

,Antecedente,Consequente,Confiança
0,Cucumber Kirby,Banana,0.621931
1,Organic Baby Spinach,Organic Strawberries,0.613571
2,Organic Raspberries,Organic Strawberries,0.612274
3,Yellow Onions,Banana,0.591476
4,Organic Garlic,Organic Baby Spinach,0.587243
5,Organic Baby Spinach,Bag of Organic Bananas,0.587166
6,Organic Blueberries,Organic Strawberries,0.580165
7,Organic Raspberries,Bag of Organic Bananas,0.569224
8,Organic Zucchini,Organic Baby Spinach,0.563903
9,Organic Hass Avocado,Bag of Organic Bananas,0.562500


**Confidence** (Confiança)

Essa medida define a probabilidade de ocorrência de consequentes no carrinho, uma vez que o carrinho já possui os antecedentes. 

Essa medida é usada para responder à pergunta: De todas as transações contendo {Manteiga}, quantas também tinham {Pão}? Podemos dizer que é de conhecimento comum que {Manteiga} → {Pão} deve ser uma regra de alta confiança. 

Tecnicamente, a confiança é a probabilidade condicional de ocorrência do consequente dado o antecedente. 

Matematicamente:

![title](imagens/confidence.png)

Não importa o que você tenha no antecedente para um consequente tão frequente. A confiança para uma regra de associação com um consequente muito frequente sempre será alta.

## Conclusão

Podemos ver através das regras de associação acima, a relação entre antecedente e consequente e o nível de confiança em ordem decrescente

Ex: 

1) Quem comprou Pepino Kirby poderá adquirir Banana com um nível de confiança de 62,1931 % 

2) O Cliente que comprou Espinafre bebê orgânico também poderá adquirir Morangos Orgânicos com um nível de confiança de 61,3571 % 

3) Quem comprou Framboesas Orgânicas poderá adquirir Morangos Orgânicos com um nível de confiança de 61,2274 % 

4) O Cliente que comprou Cebolas Amarelas poderá também adquirir Banana com um nível de confiança de 59,1476 % 


# Fim